In [ ]:
# Implied Volatility Finder


In [ ]:
from libs import my_functions
import libs.universe as tickers
import pandas as pd
import pprint


In [ ]:
start = '2023-09-01'
end = '2023-09-07'
tickers_list = tickers.Universe


In [ ]:
result = my_functions.Alpaca_Markets.get_stock_data(tickers_list, '23Hour', start, end)
result


In [ ]:
iv_result = my_functions.calculate_volatility(result, tickers_list, '2023-06-01', end)
iv_resultsorted_iv = iv_result.sort_values(ascending = False).head(20)
iv_resultsorted_iv


In [ ]:
polygon_response = my_functions.Polygon_Markets.get_stock_data('AAPL', 1, 'day', '2023-09-01', '2023-09-12', 50000)
polygon_response


In [ ]:
polygon_options = my_functions.Polygon_Markets.get_options_data('F', '2023-09-18', '2023-10-30', 20, 150)


In [ ]:
opt = pprint.pprint(polygon_options)

In [ ]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json

load_dotenv()

polygon_key = os.getenv("POLYGON_API_KEY")

url = 'https://api.polygon.io/v3/snapshot/options/'
url += 'EVRI'
url+= f'?apiKey={polygon_key}'

res = requests.get(url)
res_formatted = res.json()
pprint(res_formatted)


In [ ]:
exp_date = res_formatted['results'][0]['details']['expiration_date']
strike = res_formatted['results'][0]['details']['strike_price']
type = res_formatted['results'][0]['details']['contract_type']
print(f'Expiration Date: {exp_date}\nStrike: {strike}\nContract Type: {type}')


In [ ]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json
from libs import universe
import pandas as pd
import logging
import time

load_dotenv()

symbol_list = universe.Big_Guys
contract_type = 'PUT'
count = 4
to_date = '2023-10-27'

consumer_key = os.getenv('TDAMERITRADE_CONSUMER_KEY')
 
# Number of symbols to check per minute (TDA has a limit of calls per minute)
max_sym_limit = 100
# Number of seconds to pause and wait once max_sym_limit has been reached
seconds_to_pause = 65

data = {}
my_data = []

# Create logging object
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s %(message)s', filename='./exceptions.log', level=logging.INFO)

for i in range(len(symbol_list)):
    # Check if first symbol in list and ignore time delay
    if i != 0:
        # Check if maximum symbol limit is reached and delay GET Request
        if i % max_sym_limit == 0:
            time.sleep(seconds_to_pause)
    
    url = 'https://api.tdameritrade.com/v1/marketdata/chains?'
    url += f'apikey={consumer_key}'
    url += f'&symbol={symbol_list[i]}'
    url += f'&contractType={contract_type}'
    url += f'&strikeCount={count}'
    url += f'&toDate={to_date}'

    try:
        td_resp = requests.get(url)
    except Exception as e:
         logger.info(f'\nIssue is with response from GET request. {e}\nSymbol is:\n{symbol_list[i]}\n\n')

    symbol_data = td_resp.content.decode('ASCII')
    res_again = json.loads(symbol_data)
    try:
        if res_again["status"] == 'SUCCESS':
            data["symbol"] = res_again['symbol']
            data["underlying price"] = float(res_again['underlyingPrice'])
            data["strike price"] = float(list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0])
            data["volatility"] = float(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['volatility'])

            my_data.append(data.copy())
    except Exception as e:
            # Log exceptions and continue to next symbol
            logger.info(f'\nIssue that was thrown is related to data "status" check or data assignment to dictionary:\n{e}\nSymbol is:\n{symbol_list[i]}\n\n')
            continue



In [ ]:
data_df = pd.DataFrame(my_data).set_index('symbol')
data_df.sort_values(by=['strike price'], ascending=False, inplace=True)
data_df.head(10)



In [ ]:
data_df_filtered_vol = data_df[data_df['volatility'] > 50]
data_filtered_more = data_df_filtered_vol[data_df_filtered_vol['underlying price'] < 150.0]
data_filtered_more
